# Minecraft Texture VAE Proof of Concept

This notebook implements a Convolutional Variational Autoencoder (VAE) to learn the latent space of 16x16 Minecraft item textures using modular python files.

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from dataset import MCTextureDataset
from model import VAE
from train import train_vae
from utils import visualize_reconstruction, generate_samples, interpolate_samples, save_model, load_model, latent_arithmetic

In [ ]:
# Configuration
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 500
LATENT_DIM = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "mc_items_png"

print(f"Using device: {DEVICE}")

## Dataset Loading

In [ ]:
dataset = MCTextureDataset(DATA_DIR)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"Dataset size: {len(dataset)} images")

# Visualize dataset samples
images = next(iter(dataloader))
plt.figure(figsize=(10, 5))
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(images[i].permute(1, 2, 0).numpy())
    plt.axis('off')
plt.show()

## Training

In [ ]:
model = VAE(latent_dim=LATENT_DIM)
train_loss = train_vae(model, dataloader, EPOCHS, LEARNING_RATE, DEVICE)

In [ ]:
plt.plot(train_loss)
plt.title("Training Loss")
plt.show()

## Model Saving/Loading

In [ ]:
# save_model(model, "vae_mc_texture_dim16.pth")

# Load model
model = VAE(latent_dim=LATENT_DIM)
load_model(model, "vae_mc_texture_dim16.pth", DEVICE)

## Results

In [ ]:
visualize_reconstruction(model, dataloader, DEVICE)

In [ ]:
generate_samples(model, DEVICE, LATENT_DIM)

In [ ]:
interpolate_samples(model, dataloader, DEVICE, steps=10)

## Latent Arithmetic
Example: Iron Chestplate - Iron + Diamond = ?

In [ ]:
# Example: Iron Chestplate - Iron + Diamond
# Note: Keywords must match filenames in mc_items folder (e.g., 'iron', 'diamond')
latent_arithmetic(model, dataset, DEVICE, "iron_chestplate.png", "iron", "diamond")

In [ ]:
# Another example: Diamond Sword - Diamond + Gold = ?
latent_arithmetic(model, dataset, DEVICE, "diamond_sword.png", "diamond", "gold")